# Régression logistique à partir de zéro!

Notre but est le suivant: nous allong créer des données synthetiques bidimensionnelles qui sont linearement séparables, puis nous allons écrire un algorithme de régression logistique. C'est un tres bon, et tres instructif, exercice de python!

Commencons par creer des points en dimension 2:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

np.random.seed(12)
num_observations = 500

x1 = np.random.multivariate_normal([0, 0], [[1, .5],[.5, 1]], num_observations)
x2 = np.random.multivariate_normal([1, 4], [[1, .8],[.8, 1]], num_observations)

dataset = np.vstack((x1, x2)).astype(np.float32)
labels = np.hstack((np.zeros(num_observations),np.ones(num_observations)))

plt.figure(figsize=(6,4))
plt.scatter(dataset[:, 0], dataset[:, 1],c = labels, alpha = .4)

Notre objectif est d'utiliser une modélisation de type logistique ici. Dans ce cas:
$$P_{\rm model}(l({\bf x})=1) = \frac 1{1+\exp(-{\bf \theta} \cdot {\bf x})}~~~ \text{&}~~~ P_{\rm model}(l({\bf x})=0) = \frac {\exp(-{\bf \theta} \cdot {\bf x})}{1+\exp(-{\bf \theta} \cdot {\bf x})}$$

Maintenant, nous utilisons comme fonction de perte l'entropie croisée, et nous écrivons:
$${\rm Loss} = - \sum_{\rm dataset} \sum_{l=0,1} P_{\rm true}(x=l) \log(P_{\rm model}(x=l)) $$

Verifiez que cela donne:
$${\rm Loss} =  \sum_{\rm dataset} - y_i {\bf \theta} \cdot {\bf x}_i  + \log{(1+\exp({\bf \theta} \cdot {\bf x}_i ))} $$

Cette expression est un peu differente de celle. vue. en cours, mais elle est tout a fait equivalente. Il nous faut maintenant implementer cette fonction:

In [ ]:
def sigmoid(scores):
    return 1 / (1 + np.exp(-scores))
def log_loss(features, target, weights):
    scores = np.dot(features, weights)
    ll = #WRITE HERE THE EXPRESSION OF THE LOSS
    return -ll

Afin de réaliser l'optimisation, nous devons calculer le gradient et effectuer une montée de gradient. Ici nous avons:

$$\nabla {\rm Loss} =  \sum_{i \in \rm dataset} - y_i  {\bf x}_i  + {\bf x}_i  \frac{\exp({\bf \theta} \cdot {\bf x}_i )}{(1+\exp({\bf \theta} \cdot {\bf x}_i ))} = - \sum_{i \in \rm dataset} {\bf x^T}_i (y_i - P(l({\bf x_i})=1)) $$

Nous pouvons maintenant écrire la régression logistique et effectuer la descenet de gradient en ecrivant:

$$
{\bf \theta}^{t+1} = {\bf \theta}^{t} - \eta \nabla {\rm Loss} 
$$
ou $\eta$ est  appelle le "learning rate".

In [ ]:
def logistic_regression(features, target, num_steps, learning_rate):
#logistic regression with leanring rate learning_rate for a number of steps num_steps
#target is the true set of labels, and features is the matrix of data n * d

    weights = np.zeros(features.shape[1])##Initialization from 0
    
    for step in range(num_steps):
        scores = np.dot(features, weights)
        predictions = sigmoid(scores)

        # Update weights with gradient
        output_error_signal = ### IMPLENTEZ ICI ????????????
        gradient = ### IMPLENTEZ ICI ????????
        weights += ### IMPLENTEZ ICI ???????
        
        # Print log-likelihood from time to time
        if step % 10000 == 0:
            print (step," ",log_loss(features, target, weights))
        
    return weights

Nous utilisons l’astuce habituelle qui consiste à ajouter un "1" aux données pour pouvoir effectuer un apprentissage avec une fonction afine (ax+b) plutot que simplement lineare (ax), et nous appelons la fonction de régression:

In [ ]:
intercept = np.ones((dataset.shape[0], 1))
data_with_intercept = np.hstack((intercept, dataset))

weights = logistic_regression(data_with_intercept, labels, num_steps = 300000, learning_rate = 5e-5)

Nous pouvons maintenant tracer les prévisions à partir de notre modèle et vérifier leur qualité dans le jeu de données d'apprentissage.

In [ ]:
def line(x,a,b,c):
    return -x*b/c-a/c
def myline(x):
    return line(x,weights[0],weights[1],weights[2])

final_scores = np.dot(data_with_intercept, weights)
preds = np.round(sigmoid(final_scores))

print('Accuracy: {0}'.format((preds == labels).sum().astype(float) / len(preds)))

In [ ]:
plt.figure(figsize = (6, 4))
plt.scatter(dataset[:, 0], dataset[:, 1],
            c = (preds == labels) , alpha = .8, s = 50)
plt.plot([-3,4],[myline(-3),myline(4)])

Que se passe t-il si le num_steps est trop petit? Ou si le learning_rate est trop grand? 